In [119]:
import random

class JadwalKuliah:
    def __init__(self, kode_matkul, kode_dosen, sks_akademik, kode_ruangan, hari, jam_mulai, jam_selesai, tipe_kelas):
        self.kode_matkul = kode_matkul
        self.kode_dosen = kode_dosen
        self.sks_akademik = sks_akademik
        self.kode_ruangan = kode_ruangan
        self.hari = hari
        self.jam_mulai = jam_mulai
        self.jam_selesai = jam_selesai
        self.tipe_kelas = tipe_kelas  # 'teori' atau 'praktikum'

BOBOT_PENALTI = {
    "konflik_dosen": 15, # dosen mengajar 2/lebih kelas bersamaan
    "konflik_ruangan": 15, # kelas digunakan 2/lebih mata kuliah bersamaan
    "konflik_dosen_asisten": 15, # kelas dosen dan asistensi berjalan dalam waktu yang sama
    "beban_sks_berlebih": 10, # beban sks melebihi batas
    "weekend_class": 10, # kelas diadakan di hari Sabtu
    "konflik_istirahat": 5, # kelas diadakan di jam istirahat
    "salah_tipe_ruangan": 3, # tipe ruangan salah (praktikum di teori)
    "lewat_jam_kerja": 10, # as it says
    "tidak_sesuai_preferensi": 5 # as it says
}

# List mata kuliah
matakuliah_list = [
    {"id": "MK001", "nama": "Algoritma", "sks": 2, "tipe": "teori"},
    {"id": "MK002", "nama": "Struktur Data", "sks": 2, "tipe": "teori"},
    {"id": "MK003", "nama": "Basis Data", "sks": 3, "tipe": "praktikum"},
    {"id": "MK004", "nama": "Pemrograman", "sks": 3, "tipe": "praktikum"},
    {"id": "MK005", "nama": "Jaringan", "sks": 4, "tipe": "teori"},
    # {"id": "MK006", "nama": "Data Analist", "sks": 4, "tipe": "praktikum"},
    # {"id": "MK007", "nama": "Website Programming", "sks": 4, "tipe": "praktikum"},
    # {"id": "MK008", "nama": "Matematika", "sks": 4, "tipe": "teori"},
    # {"id": "MK009", "nama": "Bahasa", "sks": 4, "tipe": "teori"},
    # {"id": "MK010", "nama": "AI", "sks": 4, "tipe": "teori"},
]

# List dosen
dosen_list = [
    {"id": "D001", "nama": "Pak Budi", "preferensi": {"hindari_jam": [8],"hindari_hari": ["Senin"]}},
    {"id": "D002", "nama": "Bu Rina", "preferensi": {"hindari_jam": [9, 11, 12],"hindari_hari": ["Rabu"]}},
    {"id": "D003", "nama": "Pak Slamet", "preferensi": {"hindari_jam": [12],"hindari_hari": ["Jumat", "Senin", "Sabtu"]}},
]

# List ruang (teori = T, praktikum = P)
ruang_list = [
    {"id": "T101", "tipe": "teori"},
    {"id": "T102", "tipe": "teori"},
    {"id": "P201", "tipe": "praktikum"},
    {"id": "P202", "tipe": "praktikum"},
]

In [ ]:
def repair_jadwal(jadwal, dosen_list, ruang_list, hari_list, jam_list):
    # 1. Perbaiki bentrok dosen
    for i, sesi1 in enumerate(jadwal):
        for j, sesi2 in enumerate(jadwal):
            if i != j and sesi1.hari == sesi2.hari and sesi1.jam == sesi2.jam:
                if sesi1.id_dosen == sesi2.id_dosen:
                    sesi2.jam = random.choice([j for j in jam_list if j != sesi1.jam])

    # 2. Pindahkan sesi dari hari Sabtu jika dosen mengajar
    for sesi in jadwal:
        if sesi.hari == "Sabtu" and sesi.tipe_kelas == "teori":
            sesi.hari = random.choice([h for h in hari_list if h != "Sabtu"])

    # 3. Pindahkan praktikum dari ruang teori jika memungkinkan
    for sesi in jadwal:
        if sesi.tipe_kelas == "praktikum" and sesi.ruang.startswith("T"):
            ruang_praktikum = [r['id'] for r in ruang_list if r['id'].startswith("P")]
            if ruang_praktikum:
                sesi.ruang = random.choice(ruang_praktikum)

    # 4. Perbaiki beban SKS dosen
    sks_dosen = {}
    for sesi in jadwal:
        sks_dosen[sesi.id_dosen] = sks_dosen.get(sesi.id_dosen, 0) + 1

    for sesi in jadwal:
        if sks_dosen[sesi.id_dosen] > 12:
            dosen_tersedia = [d['id'] for d in dosen_list if d['id'] != sesi.id_dosen and sks_dosen.get(d['id'], 0) < 12]
            if dosen_tersedia:
                sesi.id_dosen = random.choice(dosen_tersedia)

    return jadwal

In [11]:
def generate_jadwal(matakuliah_list, dosen_list, ruang_list):
    jadwal = []

    for matkul in matakuliah_list:
        dosen = random.choice(dosen_list)
        ruang = random.choice(ruang_list)
        hari = random.choice(["Senin", "Selasa", "Rabu", "Kamis", "Jumat", "Sabtu"])
        jam_mulai = random.choice([ i for i in range(7, 19)])
        jam_selesai = jam_mulai + matkul['sks']

        sesi = JadwalKuliah(
            kode_matkul=matkul['id'],
            kode_dosen=dosen['id'],
            sks_akademik=matkul['sks'],
            kode_ruangan=ruang['id'],
            hari=hari,
            jam_mulai=jam_mulai,
            jam_selesai=jam_selesai,
            tipe_kelas=matkul['tipe']
        )
        jadwal.append(sesi)

    # 🛠️ REPAIR sebelum return
    # jadwal = repair_jadwal(jadwal, dosen_list, ruang_list, hari_list, jam_list)
    return jadwal

def generate_populasi(matakuliah_list, dosen_list, ruang_list, ukuran_populasi):
    return [generate_jadwal(matakuliah_list, dosen_list, ruang_list) for _ in range(ukuran_populasi)]

In [ ]:
# ==================================== TESTING CODE ===========================================

hari = random.choice(["Senin", "Selasa", "Rabu", "Kamis", "Jumat", "Sabtu"])
jam_mulai = random.choice([ i for i in range(7, 18)])
jam_selesai = jam_mulai + 3

print([ i for i in range(7, 19)])
print(hari, jam_mulai, jam_selesai)

if 9 in range(jam_mulai, jam_selesai) or 11 in range(jam_mulai, jam_selesai):
    print('9/11', True)

if (hari, jam_mulai) in [['Senin']]:
    print('bentrok',True)

def cetak(jadwal):
    for sesi in jadwal:
        # Data dosen saat ini
        dosen_info = next(d for d in dosen_list if d["id"] == sesi.kode_dosen)
        preferensi = dosen_info.get("preferensi", {})

        key_waktu = (sesi.hari, sesi.jam_mulai, sesi.jam_selesai)
        print(sesi.kode_matkul)
        print('  preferensi dosen', preferensi)
        print('  jadwal', key_waktu)

        if sesi.hari in preferensi.get("hindari_hari", []):
            print('    melanggar preferensi hari')
        for time in preferensi.get("hindari_jam", []):
            if time in range(sesi.jam_mulai, sesi.jam_selesai):
                print('    melanggar preferensi waktu')
                break
        
        
populasi = generate_populasi(matakuliah_list, dosen_list, ruang_list, 1)
for j in populasi:
    cetak(j)

In [13]:
def tampilkan_jadwal(jadwal):
    print("\n📅 Jadwal Mata Kuliah Terbaik:\n")
    print(f"{'MK':<8}{'Dosen':<15}{'Ruang':<8}{'Hari':<10}{'Jam Mulai':<12}{'Jam Selesai':<12}{'Tipe'}")
    print("-" * 60)
    for sesi in sorted(jadwal, key=lambda x: (x.hari, x.jam_mulai)):
        print(f"{sesi.kode_matkul:<8}{sesi.kode_dosen:<15}{sesi.kode_ruangan:<8}{sesi.hari:<10}{sesi.jam_mulai:<12}{sesi.jam_selesai:<12}{sesi.tipe_kelas}")

In [118]:
jadwal = generate_jadwal(matakuliah_list, dosen_list, ruang_list)

def hitung_fitness(jadwal):
    penalti = 0
    dosen_jadwal = {}
    ruangan_jadwal = {}
    sks_tanggungan_dosen = {}

    # COUNTER
    hitung_konflik_dosen = 0
    hitung_konflik_ruangan = 0

    for sesi in jadwal:
        # Data dosen saat ini
        dosen_info = next(d for d in dosen_list if d["id"] == sesi.kode_dosen)
        preferensi = dosen_info.get("preferensi", {})

        # 1. Beban SKS Dosen
        sks_tanggungan_dosen[sesi.kode_dosen] = sks_tanggungan_dosen.get(sesi.kode_dosen, 0) + sesi.sks_akademik
        if sks_tanggungan_dosen[sesi.kode_dosen] > 12:
            print('  sks_tangungan', sesi.kode_dosen, 'berlebih (', sks_tanggungan_dosen[sesi.kode_dosen], ')')
            penalti += (sks_tanggungan_dosen[sesi.kode_dosen] - 12) * BOBOT_PENALTI['beban_sks_berlebih']

        # 2. Dosen tidak boleh ganda
        # print('  jadwal dosen', sesi.kode_dosen, sesi.hari, sesi.jam_mulai, sesi.jam_selesai)
        if sesi.kode_dosen not in dosen_jadwal:
            dosen_jadwal[sesi.kode_dosen] = []

        for sesi_lain in dosen_jadwal[sesi.kode_dosen]:
            if sesi.hari == sesi_lain['hari']:
                # print('    jadwal dosen', sesi.kode_dosen, sesi_lain['hari'], sesi_lain['mulai'], sesi_lain['selesai'])
                if (sesi.jam_mulai in range(sesi_lain['mulai']+1, sesi_lain['selesai'])) or (sesi.jam_selesai  in range(sesi_lain['mulai']+1, sesi_lain['selesai'])):
                    print('    bentrok dosen', sesi.kode_dosen)
                    penalti += BOBOT_PENALTI['konflik_dosen']
                    hitung_konflik_dosen += 1
        
        dosen_jadwal[sesi.kode_dosen].append({'hari': sesi.hari, 'mulai': sesi.jam_mulai, 'selesai': sesi.jam_selesai})
        
        # 3. Ruangan tidak boleh digunakan lebih dari 1 matkul secara bersamaan
        # print('  jadwal ruangan', sesi.kode_ruangan, sesi.hari, sesi.jam_mulai, sesi.jam_selesai)
        if sesi.kode_ruangan not in ruangan_jadwal:
            ruangan_jadwal[sesi.kode_ruangan] = []

        for sesi_lain in ruangan_jadwal[sesi.kode_ruangan]:
            if sesi.hari == sesi_lain['hari']:
                # print('    jadwal ruangan', sesi.kode_ruangan, sesi_lain['hari'], sesi_lain['mulai'], sesi_lain['selesai'])
                if (sesi.jam_mulai in range(sesi_lain['mulai']+1, sesi_lain['selesai'])) or (sesi.jam_selesai  in range(sesi_lain['mulai']+1, sesi_lain['selesai'])):
                    print('    bentrok ruangan', sesi.kode_ruangan)
                    penalti += BOBOT_PENALTI['konflik_dosen']
                    hitung_konflik_ruangan += 1

        ruangan_jadwal[sesi.kode_ruangan].append({'hari': sesi.hari, 'mulai': sesi.jam_mulai, 'selesai': sesi.jam_selesai})

        # 4. Praktikum di ruang teori
        if sesi.tipe_kelas == "PRAKTIKUM" and next((ruang['tipe'] for ruang in ruang_list if ruang["id"] == sesi.kode_ruangan), None).upper().startswith("T"):
            penalti += BOBOT_PENALTI['salah_tipe_ruangan']

        # 5. Dosen tidak boleh Sabtu
        if sesi.hari == "Sabtu":
            print('(-10) dosen', sesi.kode_dosen, ' mengajar di hari weekend (', sesi.hari, ')')
            penalti += BOBOT_PENALTI['weekend_class']

        # 6. Praktikum dianjurkan Sabtu (pagi) (OPTIONAL)
        if sesi.tipe_kelas == "PRAKTIKUM":
            if sesi.hari != "Sabtu":
                print('(-2) hari praktikum != Sabtu')
                penalti += 2
            elif sesi.jam_selesai > 13:
                print('(-2) jam praktikum > 13')
                penalti += 2

        # 7. Preferensi dosen
        if sesi.hari in preferensi.get("hindari_hari", []):
            print('(-5) tidak sesuai preferensi hari dosen')
            penalti += BOBOT_PENALTI['tidak_sesuai_preferensi']
        for waktu in preferensi.get("hindari_jam", []):
            if waktu in range(sesi.jam_mulai, sesi.jam_selesai):
                print('(-5) tidak sesuai preferensi waktu dosen')
                penalti += BOBOT_PENALTI['tidak_sesuai_preferensi']
                break

    print(f"(-{hitung_konflik_dosen * BOBOT_PENALTI['konflik_dosen']}) Bentrok Dosen: {hitung_konflik_dosen}")
    print(f"(-{hitung_konflik_ruangan * BOBOT_PENALTI['konflik_ruangan']}) Bentrok Ruangan: {hitung_konflik_ruangan}")
    return max(0, 100 - penalti)

hitung_fitness(jadwal)

(-10) dosen D003  mengajar di hari weekend ( Sabtu )
(-5) tidak sesuai preferensi hari dosen
(-10) dosen D001  mengajar di hari weekend ( Sabtu )
(-5) tidak sesuai preferensi hari dosen
    bentrok dosen D001
  sks_tangungan D001 berlebih ( 13 )
(-10) dosen D001  mengajar di hari weekend ( Sabtu )
    bentrok dosen D003
(-5) tidak sesuai preferensi hari dosen
  sks_tangungan D001 berlebih ( 17 )
    bentrok dosen D001
(-10) dosen D001  mengajar di hari weekend ( Sabtu )
  sks_tangungan D001 berlebih ( 21 )
(-5) tidak sesuai preferensi hari dosen
  sks_tangungan D003 berlebih ( 13 )
(-45) Bentrok Dosen: 3
(-0) Bentrok Ruangan: 0


0

In [24]:
def roulette_selection(populasi, fitness_scores):
    total_fitness = sum(fitness_scores)
    pick = random.uniform(0, total_fitness)
    current = 0
    for i, score in enumerate(fitness_scores):
        current += score
        if current > pick:
            return populasi[i]

In [25]:
def crossover(parent1, parent2):
    titik = random.randint(1, len(parent1) - 2)
    child1 = parent1[:titik] + parent2[titik:]
    child2 = parent2[:titik] + parent1[titik:]
    return child1, child2

In [26]:
def mutasi(jadwal, ruang_list, slot_waktu, peluang=0.05):
    for sesi in jadwal:
        if random.random() < peluang:
            sesi.hari, sesi.jam = random.choice(slot_waktu)
            sesi.ruang = random.choice([r['id'] for r in ruang_list if r['tipe'] == sesi.tipe_kelas])
    return jadwal

In [ ]:
def genetic_algorithm(matakuliah_list, dosen_list, ruang_list, ukuran_populasi=50, generasi=100):
    populasi = generate_populasi(matakuliah_list, dosen_list, ruang_list, ukuran_populasi)

    for gen in range(generasi):
        fitness_scores = [hitung_fitness(j) for j in populasi]
        next_gen = []

        while len(next_gen) < ukuran_populasi:
            parent1 = roulette_selection(populasi, fitness_scores)
            parent2 = roulette_selection(populasi, fitness_scores)
            child1, child2 = crossover(parent1, parent2)
            child1 = mutasi(child1, ruang_list)
            child2 = mutasi(child2, ruang_list)
            next_gen.extend([child1, child2])

        populasi = next_gen[:ukuran_populasi]
        best = max(fitness_scores)
        print(f"Generasi {gen}: Fitness terbaik = {best}")
        debug_penalti(populasi)

    # Ambil hasil terbaik
    fitness_scores = [hitung_fitness(j) for j in populasi]
    best_jadwal = populasi[fitness_scores.index(max(fitness_scores))]
    return best_jadwal

In [ ]:
best_schedule = genetic_algorithm(
    matakuliah_list, 
    dosen_list, 
    ruang_list, 
    slot_waktu, 
    ukuran_populasi=30, 
    generasi=50
)